In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 9.5 MB/s eta 0:00:00


In [ ]:
import os
import SimpleITK as sitk

def convert_all_dicom_series_to_mhd(root_dir, output_dir):
    for subdir, _, _ in os.walk(root_dir):
        print("Processing: ",subdir)
        dicom_files = [f for f in os.listdir(subdir) if f.endswith(".dcm")]
        if dicom_files:
            reader = sitk.ImageSeriesReader()
            dicom_series = reader.GetGDCMSeriesFileNames(subdir)
            reader.SetFileNames(dicom_series)
            image = reader.Execute()

            # Save the image in MHD format with the subfolder name
            subfolder_name = os.path.basename(subdir)
            output_mhd_file = os.path.join(output_dir, f"{subfolder_name}.mhd")
            sitk.WriteImage(image, output_mhd_file)

# Example usage
root_dir = "/content/drive/MyDrive/Dateset/DICOM"
output_dir = "/content/drive/MyDrive/Elcap_Resource"
convert_all_dicom_series_to_mhd(root_dir, output_dir)

Processing:  /content/drive/MyDrive/Dateset/DICOM
Processing:  /content/drive/MyDrive/Dateset/DICOM/1
Processing:  /content/drive/MyDrive/Dateset/DICOM/10
Processing:  /content/drive/MyDrive/Dateset/DICOM/11
Processing:  /content/drive/MyDrive/Dateset/DICOM/12
Processing:  /content/drive/MyDrive/Dateset/DICOM/13
Processing:  /content/drive/MyDrive/Dateset/DICOM/14
Processing:  /content/drive/MyDrive/Dateset/DICOM/15
Processing:  /content/drive/MyDrive/Dateset/DICOM/16
Processing:  /content/drive/MyDrive/Dateset/DICOM/17
Processing:  /content/drive/MyDrive/Dateset/DICOM/18
Processing:  /content/drive/MyDrive/Dateset/DICOM/19
Processing:  /content/drive/MyDrive/Dateset/DICOM/2
Processing:  /content/drive/MyDrive/Dateset/DICOM/20
Processing:  /content/drive/MyDrive/Dateset/DICOM/21
Processing:  /content/drive/MyDrive/Dateset/DICOM/22
Processing:  /content/drive/MyDrive/Dateset/DICOM/23
Processing:  /content/drive/MyDrive/Dateset/DICOM/24
Processing:  /content/drive/MyDrive/Dateset/DICOM/2

In [1]:
import os
import pandas as pd

def combine_annotation_csv(annotation_folder, output_csv):
    combined_data = pd.DataFrame(columns=["seriesuid", "coordX", "coordY", "coordZ", "diameter_mm"])

    for filename in os.listdir(annotation_folder):
        if filename.endswith(".csv"):
            filepath = os.path.join(annotation_folder, filename)
            seriesuid = os.path.splitext(filename)[0]

            try:
                df = pd.read_csv(filepath)

                # Check if the DataFrame is empty
                if df.empty:
                    print(f"Warning: {filename} is empty. Skipping...")
                    continue

                # Extract X, Y, Z coordinates from the second, third, and fourth columns
                coordX = df.iloc[:, 1]
                coordY = 512 - df.iloc[:, 2]
                coordZ = df.iloc[:, 3]

                # Create a DataFrame with the desired format
                temp_df = pd.DataFrame({
                    "seriesuid": [seriesuid] * len(df),
                    "coordX": coordX,
                    "coordY": coordY,
                    "coordZ": coordZ,
                    "diameter_mm": 5
                })

                combined_data = pd.concat([combined_data, temp_df], ignore_index=True)

            except pd.errors.EmptyDataError:
                print(f"Warning: {filename} is empty or could not be parsed. Skipping...")

    combined_data.to_csv(output_csv, index=False)

# Example usage
annotation_folder = "/content/drive/MyDrive/Dateset/Annotations"
output_csv = "/content/drive/MyDrive/Elcap_Resource/combined_annotations.csv"
combine_annotation_csv(annotation_folder, output_csv)